# System building: Protein in Membrane with Ligand

In this tutorial, we will showcase how to build a protein ligand system for simulating binding. The sample system is Trypsin (the protein) and benzamidine (the ligand).

Let's start by doing some imports and definitions:

In [1]:
from htmd.ui import *
from htmd.home import home
from os.path import join
config(viewer='webgl')
datadir = home(dataDir='building-protein-ligand')

Loaded Config for HTMD @ Acellera Office

HTMD Config file /shared/acellera/conf.py executed.

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. 
https://dx.doi.org/10.1021/acs.jctc.6b00049
Documentation: http://software.acellera.com/
To update: conda update htmd -c acellera -c psi4

You are on the latest HTMD version (unpackaged : /shared/alberto/Projects/REPOS/htmd/htmd).



## Load the protein-ligand complex

One can obtain the protein-ligand complex from the PDB database (ID:3PTB). The complex is already available in the data distributed with HTMD and either one could be used:

In [2]:
# One can download it directly from the RCSB servers
prot = Molecule('3PTB')
# Or use the pdb file found in the HTMD data directory
prot = Molecule(join(datadir, 'trypsin.pdb'))
prot.renumberResidues()

2018-04-23 14:24:20,408 - htmd.molecule.readers - INFO - Using local copy for 3PTB: /shared/alberto/Projects/REPOS/htmd/htmd/data/pdb/3ptb.pdb


In [3]:
prot.view()

A Jupyter Widget

## Clean the structures

The PDB crystal structure contains the protein as well as water molecules, a calcium ion and a ligand. Here we will start by removing the ligand from the protein Molecule as we will add it later to manipulate it separately.

In [4]:
prot.remove('resname BEN')

2018-04-23 14:24:20,806 - htmd.molecule.molecule - INFO - Removed 9 atoms. 1692 atoms remaining in the molecule.


array([1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638], dtype=int32)

## Preparing the protein

In this step, we prepare the protein for simulation by adding hydrogens, setting the protonation states, and optimizing the protein (more details on the protein preparation tutorial):

In [5]:
prot = proteinPrepare(prot, pH=7.0)

2018-04-23 14:24:20,923 - propka - INFO - No pdbfile provided
2018-04-23 14:24:23,118 - htmd.builder.preparation - WARNING - The following residue has not been optimized: CA
2018-04-23 14:24:28,369 - htmd.builder.preparationdata - INFO - The following residues are in a non-standard state: CYS     6  A (CYX), HIS    22  A (HIE), CYS    24  A (CYX), HIS    39  A (HIP), CYS    40  A (CYX), HIS    72  A (HID), CYS   108  A (CYX), CYS   115  A (CYX), CYS   136  A (CYX), CYS   147  A (CYX), CYS   161  A (CYX), CYS   172  A (CYX), CYS   182  A (CYX), CYS   196  A (CYX), CYS   209  A (CYX)
2018-04-23 14:24:28,429 - htmd.builder.preparationdata - WARNING - Dubious protonation state: the pKa of 4 residues is within 1.0 units of pH 7.0.
2018-04-23 14:24:28,432 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    HIS    39  A (pKa= 7.46)
2018-04-23 14:24:28,433 - htmd.builder.preparationdata - WARNING - Dubious protonation state:    GLU    51  A (pKa= 6.10)
2018-04-23 14:24:28,

## Define segments

To build a system in HTMD, we need to separate the chemical molecules into separate segments. This prevents the builder from accidentally bonding different chemical molecules and allows us to add caps to them.

In [6]:
prot = autoSegment(prot, sel='protein')
prot.set('segid', 'W', sel='water')
prot.set('segid', 'CA', sel='resname CA')

Center the protein to the origin

In [7]:
prot.center()

## Let's work on the ligand!

Load the ligand from the HTMD data:

In [8]:
ligand = Molecule(join(datadir, 'benzamidine.mol2'))

2018-04-23 14:24:29,016 - htmd.molecule.readers - WARNING - Element of atom ID 1 could not be automatically guessed from its MOL2 atomtype (ca).
2018-04-23 14:24:29,017 - htmd.molecule.readers - WARNING - Element of atom ID 2 could not be automatically guessed from its MOL2 atomtype (ca).
2018-04-23 14:24:29,018 - htmd.molecule.readers - WARNING - Element of atom ID 3 could not be automatically guessed from its MOL2 atomtype (ca).
2018-04-23 14:24:29,019 - htmd.molecule.readers - WARNING - Element of atom ID 4 could not be automatically guessed from its MOL2 atomtype (ca).
2018-04-23 14:24:29,020 - htmd.molecule.readers - WARNING - Element of atom ID 5 could not be automatically guessed from its MOL2 atomtype (ca).
2018-04-23 14:24:29,020 - htmd.molecule.readers - WARNING - Element of atom ID 6 could not be automatically guessed from its MOL2 atomtype (ca).
2018-04-23 14:24:29,021 - htmd.molecule.readers - WARNING - Element of atom ID 7 could not be automatically guessed from its MOL2 

Let's center the ligand and visualize it:

In [9]:
ligand.center()
ligand.view()

A Jupyter Widget

In [10]:
# We can give a convenient segid and resname to the ligand
# The resname should be MOL to match the parameters in the
# rtf and prm files.
ligand.set('segid','L')
ligand.set('resname','MOL')

But the ligand is now located inside the protein...
We would like the ligand to be:

* At a certain distance from the protein
* Rotated randomly, to provide different starting conditions

## Let's randomize the ligand position

In [11]:
ligand.rotateBy(uniformRandomRotation())

This took care of the ligand rotation around its own center. 
We still need to position it far from the protein.
First, find out the radius of the protein:

<center>![maxdist](http://pub.htmd.org/tutorials/system-building-protein-ligand/maxdist.png)</center>

In [12]:
from htmd.molecule.util import maxDistance
D = maxDistance(prot, 'all')
print(D)

28.832803412


In [13]:
D += 10
# Move the ligand 10 Angstrom away from the furthest protein atom in X dimension
ligand.moveBy([D, 0, 0])  
# rotateBy rotates by default around [0, 0, 0]. Since the ligand has been moved
# away from the center it will be rotated in a sphere of radius D+10 around [0, 0, 0]
ligand.rotateBy(uniformRandomRotation())

### Mix it all together

In [14]:
mol = Molecule(name='combo')
mol.append(prot)
mol.append(ligand)
mol.reps.add(sel='protein', style='NewCartoon', color='Secondary Structure')
mol.reps.add(sel='resname MOL', style='Licorice')
mol.view()

A Jupyter Widget

## Solvate

> Water is the driving force of all nature. --Leonardo da Vinci

<center>![waterbox](http://pub.htmd.org/tutorials/system-building-protein-ligand/waterbox.png)</center>

In [15]:
# We solvate with a larger box to fully solvate the ligand
DW = D + 5
smol = solvate(mol, minmax=[[-DW, -DW, -DW], [DW, DW, DW]])
smol.reps.add(sel='water', style='Lines')
smol.view()

2018-04-23 14:24:29,688 - htmd.builder.solvate - INFO - Using water pdb file at: /shared/alberto/Projects/REPOS/htmd/htmd/builder/wat.pdb
2018-04-23 14:24:30,617 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2
Solvating: 100%|██████████| 8/8 [00:06<00:00,  1.16it/s]
2018-04-23 14:24:39,320 - htmd.builder.solvate - INFO - 20144 water molecules were added to the system.


A Jupyter Widget

## Available CHARMM forcefields

In [16]:
charmm.listFiles()

---- Topologies files list: /shared/alberto/Projects/REPOS/htmd/htmd/builder/charmmfiles/top/ ----
top/top_all22_prot.rtf
top/top_all22star_prot.rtf
top/top_all35_ethers.rtf
top/top_all36_carb.rtf
top/top_all36_cgenff.rtf
top/top_all36_lipid.rtf
top/top_all36_na.rtf
top/top_all36_prot.rtf
top/top_water_ions.rtf
---- Parameters files list: /shared/alberto/Projects/REPOS/htmd/htmd/builder/charmmfiles/par/ ----
par/par_all22_prot.prm
par/par_all22star_prot.prm
par/par_all35_ethers.prm
par/par_all36_carb.prm
par/par_all36_cgenff.prm
par/par_all36_lipid.prm
par/par_all36_na.prm
par/par_all36_prot.prm
par/par_all36_prot_mod.prm
par/par_water_ions.prm
---- Stream files list: /shared/alberto/Projects/REPOS/htmd/htmd/builder/charmmfiles/str/ ----
str/carb/toppar_all36_carb_glycolipid.str
str/carb/toppar_all36_carb_glycopeptide.str
str/carb/toppar_all36_carb_model.str
str/lipid/toppar_all36_lipid_bacterial.str
str/lipid/toppar_all36_lipid_cardiolipin.str
str/lipid/toppar_all36_lipid_cholesterol.

## Build and ionize using CHARMM

In [17]:
topos_charmm  = charmm.defaultTopo() + [join(datadir, 'benzamidine.rtf')]
params_charmm = charmm.defaultParam() + [join(datadir, 'benzamidine.prm')]

bmol_charmm = charmm.build(smol, topo=topos_charmm, param=params_charmm, outdir='./build_charmm')

2018-04-23 14:24:46,109 - htmd.builder.charmm - INFO - Writing out segments.
2018-04-23 14:24:52,191 - htmd.builder.builder - INFO - 6 disulfide bonds were added


Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 24, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 40, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 115, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 182, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 147, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 161, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 108, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 209, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 172, insertion: '', segid: 'P0'>
                   and: Un

2018-04-23 14:24:52,714 - htmd.builder.charmm - INFO - Starting the build.
2018-04-23 14:24:53,870 - htmd.builder.charmm - WARNING - Failed to set coordinates for 8 atoms.
2018-04-23 14:24:53,872 - htmd.builder.charmm - WARNING - Failed to guess coordinates for 3 atoms due to bad angles.
2018-04-23 14:24:53,873 - htmd.builder.charmm - WARNING - Poorly guessed coordinates for 15 atoms.
2018-04-23 14:24:53,874 - htmd.builder.charmm - WARNING - Please check /shared/alberto/Projects/GithubIssues/htmd_653/build_charmm/log.txt for further information.
2018-04-23 14:24:53,875 - htmd.builder.charmm - INFO - Finished building.
2018-04-23 14:24:57,100 - htmd.builder.ionize - INFO - Adding 10 anions + 0 cations for neutralizing and 0 ions for the given salt concentration.
2018-04-23 14:25:02,694 - htmd.builder.charmm - INFO - Writing out segments.
2018-04-23 14:25:10,044 - htmd.builder.charmm - INFO - Starting the build.
2018-04-23 14:25:11,204 - htmd.builder.charmm - WARNING - Failed to set coor

## Available AMBER forcefields

In [18]:
amber.listFiles()

---- Forcefield files list: /shared/alberto/miniconda3/dat/leap/cmd/ ----
leaprc.constph
leaprc.DNA.bsc1
leaprc.DNA.OL15
leaprc.ff14SB.redq
leaprc.ffAM1
leaprc.ffPM3
leaprc.gaff
leaprc.gaff2
leaprc.GLYCAM_06EPb
leaprc.GLYCAM_06j-1
leaprc.lipid14
leaprc.modrna08
leaprc.phosaa10
leaprc.protein.fb15
leaprc.protein.ff03.r1
leaprc.protein.ff03ua
leaprc.protein.ff14SB
leaprc.protein.ff14SBonlysc
leaprc.protein.ff15ipq
leaprc.protein.ff15ipq-vac
leaprc.RNA.OL3
leaprc.RNA.YIL
leaprc.water.opc
leaprc.water.spce
leaprc.water.spceb
leaprc.water.tip3p
leaprc.water.tip4pew
leaprc.xFPchromophores
---- OLD Forcefield files list: /shared/alberto/miniconda3/dat/leap/cmd/ ----
oldff/leaprc.DNA.bsc0
oldff/leaprc.ff02
oldff/leaprc.ff02pol.r0
oldff/leaprc.ff02pol.r1
oldff/leaprc.ff02polEP.r0
oldff/leaprc.ff02polEP.r1
oldff/leaprc.ff03
oldff/leaprc.ff10
oldff/leaprc.ff14ipq
oldff/leaprc.ff14SB
oldff/leaprc.ff84
oldff/leaprc.ff86
oldff/leaprc.ff94
oldff/leaprc.ff94.nmr
oldff/leaprc.ff96
oldff/leaprc.ff98
old

## Build and ionize using AMBER

In [19]:
topos_amber = amber.defaultTopo()
frcmods_amber = amber.defaultParam() + [join(datadir, 'benzamidine.frcmod')]

In [20]:
bmol_amber = amber.build(smol, topo=topos_amber, param=frcmods_amber, outdir='./build_amber')

2018-04-23 14:25:24,677 - htmd.builder.amber - INFO - Starting the build.
2018-04-23 14:25:36,308 - htmd.builder.amber - INFO - Finished building.
2018-04-23 14:25:39,552 - htmd.builder.ionize - INFO - Adding 9 anions + 0 cations for neutralizing and 0 ions for the given salt concentration.
2018-04-23 14:25:44,911 - htmd.builder.builder - WARNING - Segments ['P0'] contain both protein and non-protein atoms. Please assign separate segments to them or the build procedure might fail.
2018-04-23 14:25:46,114 - htmd.builder.amber - INFO - Detecting disulfide bonds.
2018-04-23 14:25:46,136 - htmd.builder.builder - INFO - 6 disulfide bonds were added


Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 24, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 40, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 115, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 182, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 147, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 161, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 108, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 209, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 172, insertion: '', segid: 'P0'>
                   and: Un

2018-04-23 14:25:49,473 - htmd.builder.amber - INFO - Starting the build.
2018-04-23 14:26:00,792 - htmd.builder.amber - INFO - Finished building.
2018-04-23 14:26:03,401 - htmd.molecule.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.


## Visualize

The built system can be visualized (with waters hidden to be able to visualize the inserted ions):

In [21]:
bmol_charmm.view(sel='not water')

A Jupyter Widget

In [22]:
bmol_amber.view(sel='not water')

2018-04-23 14:26:41,289 - htmd.molecule.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.


A Jupyter Widget

The `bmol` is a `Molecule` object that contains the built system, but the full contents to run a simulation are located in the `outdir` (`./build` in this case).